# Unittests - mercs/algo/inference

## Imports

In [1]:
# Standard imports
import os
import sys
from os.path import dirname


# Custom imports
root_directory = dirname(os.getcwd())
for dname in {'src'}:
    sys.path.insert(0, os.path.join(root_directory, dname))

from mercs.algo.inference import *

## Actual Tests

In [2]:
s = {'type': 'DT'}

In [3]:
induce_clf(s)

NameError: name 'induce_clf' is not defined

In [12]:
def test_induce_clf():
    s = {'type': 'DT'}
    
    clf  = induce_clf(s)
    
    assert isinstance(clf, DecisionTreeClassifier)

In [14]:
test_induce_clf()

In [18]:
s = {'type': 'DT'}
is_nominal = np.array([0,0,1,1])
m_targ = [0]

obs = induce_model(s, is_nominal, m_targ)

In [21]:
obs

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [22]:
assertInstance

NameError: name 'assertInstance' is not defined

In [32]:
def test_induce_model():
    s = {'type': 'DT'}
    is_nominal = np.array([0,0,1,1])
    
    m_targ_1 = [0]
    m_targ_2 = [2]
    m_targ_3 = [2,3]
    m_targ_4 = [0,1]
    m_targ_5 = [1,2]

    obs_1 = induce_model(s, is_nominal, m_targ_1)
    obs_2 = induce_model(s, is_nominal, m_targ_2) 
    obs_3 = induce_model(s, is_nominal, m_targ_3) 
    obs_4 = induce_model(s, is_nominal, m_targ_4) 
    
    assert isinstance(obs_1, DecisionTreeRegressor)
    assert isinstance(obs_2, DecisionTreeClassifier)
    assert isinstance(obs_3, DecisionTreeClassifier)
    assert isinstance(obs_4, DecisionTreeRegressor)
    
    try:
        obs_5 = induce_model(s, is_nominal, m_targ_5)
    except TypeError:
        pass

In [37]:
test_induce_model()

In [38]:
x = np.array([1,2,3])

In [41]:
type(x)

numpy.ndarray

In [44]:
is_nominal = np.array([1,2,3])

In [47]:
assert isinstance(is_nominal, list)

AssertionError: 